<a href="https://colab.research.google.com/github/emerenan/xlsx_validation/blob/main/hu_validations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re
from openpyxl import Workbook, styles
from openpyxl.styles import PatternFill, Font
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule

def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist


Doing all checks in Towerdb File

In [10]:
"""Defining variables which is gonna be reusable in checks"""
tw_index = 'code'
tw_doer = 'date of equipment removal'
tw_status = 'active / not active'
tw_bts = 'bts_site'
tw_bill = 'ready for active installation ("rfai") date'
tw_wip = 'wip_site'
#tw_decom = 'Decommissioned sites'
tw_critical = 'critical site'


msa_index ='code'
msa_doer = 'date of equipment removal'
msa_status = 'active / not active'
msa_bts = 'bts_site'
msa_bill = 'ready for active installation ("rfai") date'
msa_wip = 'wip_site'
#msa_decom = 'Decommissioned sites'
msa_critical = 'critical site'

In [22]:
path_msa = '/content/TowerDB_Hungary_20210731 (1).csv'
msa = pd.read_csv(path_msa, encoding='latin')

msa_cols = lower_str(list(msa.columns))
msa.columns = msa_cols
msa = msa.rename(columns={
    'additional information \x96 room configuration (including container information)': 'additional information - room configuration (including container information)'
})

path_tw = '/content/TowerDB_Hungary_20210831.csv'
towerdb = pd.read_csv(path_tw, encoding='latin').fillna('')
towerdb.columns = lower_str(list(towerdb.columns))
towerdb = towerdb.rename(columns={
    'additional information \x96 room configuration (including container information)': 'additional information - room configuration (including container information)'
})

towerdb = towerdb.fillna('')

#Check and fit date values
date_cols = ['first_active_sharing_start_date','first_active_sharing_end_date','ready for active installation ("rfai") date', 'date of equipment removal']

for i in date_cols:
    towerdb[i] = [dt if re.match(r"[0-9]{1,2}[\_|\-|\/][0-9]{1,2}[\_|\-|\/][0-9]{1,4}", dt) or dt in (None, '') or dt.isspace() else f"{dt:%d/%m/%Y}" for dt in towerdb[i]]

#Replace Invalid values
replace_cols = [
    'confirmed skylon scope',
    'code',
    'categorization by transmission sys',
    'categorization by transmission sys (subcluster)',
    'categorization by site type',
    'additional information - room configuration (including container information)',
    'type of air cooling',
    'core site type ',
    'macro site - transmission hub site',
    'macro site - transmission hub site with/without shelters',
    'active sharing arrangements involving the operator',
    'critical site',
    'strategic site',
    'telenor tenant',
    'telekom tenant',
    'power supply for tenants',
    'energy category',
    'wip_site',
    'bts_site',
    'strategic_site_bucket',
    'critical_site_beyond_10',
    'subsequent_sharing_arrangement',
    'sites_as_metered_estimated']
for i in replace_cols:
    towerdb[i] = towerdb[i].replace([ 'N/A', 'n/a', '0', ' '], '')

towerdb.to_csv('/content/TowerDB_Hungary_20210831_test.csv', index=False, encoding='latin')
towerdb.head(3)

,celltracker skylon scope,confirmed skylon scope start,confirmed skylon scope end,active / not active,confirmed skylon scope,code,code1,third site id,site name,macro region,province,municipality,inhabitants,address,ground register,altitude,latitude,longitude,categorization by transmission sys,categorization by transmission sys (subcluster),das system,active/passive das,das ownership (fully / partially),vertical passive structure owner,vertical passive structure owner classification,das owner,other categorization for internal use,categorization by site type,categorization by inhabitants,type of structure,other internal categorization 1,other internal categorization 2 (sitetypemain),other internal categorization 3 (sitetypemain_v1),other internal categorization 4 (sitetypemain_v2),other internal categorization 5 (sitetypemain_archive),other internal categorization 6 (sitetypeminor),other internal categorization 7 (sitetypeminor_archive),other internal categorization 8,other internal categorization 9 (for the 1 by 1 reviewed sites only),technology vod,...,annual hosting digi in huf,annual hosting digi in eur,annual energy digi,annual maintenance fee digi,other services fee digi,total revenues digi in huf,total revenues digi in eur,residual duration digi,maturity clusters digi,otmos,annual hosting otmos in huf,annual hosting otmos in eur,annual energy otmos,annual maintenance fee otmos,other services fee otmos,total revenues otmos in huf,total revenues otmos in eur,residual duration otmos,maturity clusters otmos,total # of 3rd party tenants,annual hosting fee from 3rd party tenants in huf,annual hosting fee from 3rd party tenants in eur,annual energy fee from 3rd party tenants,annual maintenance fee from 3rd party tenants,other services fee from 3rd party tenants,total hosting fee & services from 3rd party tenants in huf,total hosting fee & services from 3rd party tenants in eur,waighted average residual duration,adjusted status,adjustes scope,wip_site,bts_site,strategic_site_bucket,critical_site_beyond_10,subsequent_sharing_arrangement,first_active_sharing_start_date,first_active_sharing_end_date,active sharing deployment types,sites_as_metered_estimated,date of equipment removal
0,In scope,01/04/2017,,In Service,In scope,1001,,,BUDAPESTNYUGATI TER,Pest,Budapest,Budapest,1 735 711,Nyugati tér 5.,25054/3,105,47.51097712,19.05516864,Macro,Standard,No,,,Vodafone vertical,Vodafone,,Macro,RTT,Urban,Rooftop structure,Pitched roof,Pitched roof,Pitched roof,Roof,Lapostetö/Magastetö,Pitched roof,magastetö,Pitched roof,,2G3G4G,...,0,0,,,,0,0,0,exclude,0,0,0,,,,0,0,0,exclude,0,0,0,,,,0,0,0,,,No,No,Non Strategic,Non Critical,,,,,Estimated Model,
1,In scope,01/04/2017,,In Service,In scope,1002,,,DEAK tér,Pest,Budapest,Budapest,1 735 711,Bajcsy Zsilinszky u. 9.,29264/2,105,47.49906804,19.05508727,Macro,Standard,No,,,Vodafone vertical,Vodafone,,Macro,RTT,Urban,Rooftop structure,Flat roof,Flat roof,Flat roof,Roof,Lapostetö/Magastetö,Flat roof,lapostetö,Flat roof,,2G3G4G,...,0,0,,,,0,0,0,exclude,0,0,0,,,,0,0,0,exclude,0,0,0,,,,0,0,0,,,No,No,Non Strategic,Non Critical,,,,,Metered Model,
2,In scope,01/04/2017,,In Service,In scope,1004,,,BUDAPEST-BKV,Pest,Budapest,Budapest,1 735 711,Szabó Ervin tér 2.,36750/3,103,47.48905593,19.06418919,Macro,Standard,No,,,Vodafone,Vodafone,,Macro,RTT,Urban,Rooftop structure,Flat roof,Flat roof,Flat roof,Roof,Lapostetö/Magastetö,Flat roof,lapostetö,Flat roof,lapostetö,2G3G4G,...,0,0,,,,0,0,0,exclude,0,0,0,,,,0,0,0,exclude,0,0,0,,,,0,0,0,,,No,No,Non Strategic,Non Critical,,,,,Metered Model,


In [ ]:
towerdb[]

In [ ]:
"""Check Columns Received""" 
def check_columns_received(df, bill_cols):
    twdb_col = lower_str(list(df.columns))
    col_miss = [i for i in bill_cols if i not in twdb_col]
    """
    for i in bill_cols:
        if i not in twdb_col:
            col_miss.append(i)"""
    df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'], index=range(len(col_miss)))
    return df_col_missing

df_cols = check_columns_received(towerdb, list(msa.columns))
df_cols
#No columns missing

,Column(s) Missing


First Check - Dates Formats (dd/mm/YYYY)

Columns: Date of equipment removal

In [ ]:
"""You need to convert all values in cols for string format to check"""
"""CSV FIles doesn't need convert, only fill na values"""
def check_date_columns(df, df_index,status_col,columns):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    from collections import defaultdict

    df_dates = df[columns].fillna('')
    df_dates = df_dates.to_dict(orient='list')
    #df_dates['sites'] = df_dates[df_index]
    #df_dates = df_dates.set_index('sites')
    new_dic = defaultdict(list)

    date_format = re.compile(r"[0-9]{1,2}[\_|\-|\/][0-9]{1,2}[\_|\-|\/][0-9]{1,4}")

    for column in set(df_dates.keys()):
        for value in df_dates[column]:
            if date_format.match(value) == None:
                if value in (None, '', 'nan'):
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorret picklist value: {value}')
            else:
                new_dic[column].append('Ok')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok', np.nan)
    df_errors[df_index] = [i for i in df[df_index]]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()

    df = df[[df_index, status_col]]
    df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors[[status_col]+ df_errors.columns[:-1].tolist()]
    df_errors = df_errors.reset_index()
    return df_errors

actives_1 = towerdb[towerdb[tw_status]=='In Service']
no_actives_1 = towerdb[~(towerdb[tw_status]=='In Service')]

#Checking columns for errors
actives_dates_errors = check_date_columns(actives_1, tw_index, tw_status, [tw_bill]) 
# Actives sites with blank billing trigger date
no_actives_dates_errors = check_date_columns(no_actives_1, tw_index, tw_status, [tw_doer]) 

print(actives_dates_errors)
print('\n')
print(no_actives_dates_errors)


Thirth - Check Picklist values All sites

Do this check in all sites

Check the picklist for each case

In [13]:
def check_picklist_v1(df,df_index,df_status, picklist_dict):
    from collections import defaultdict

    df_cols = list(picklist_dict.keys())
    df_picklist = df[df_cols].fillna('')
    df_picklist = df_picklist.to_dict(orient='list')

    new_dic = defaultdict(list)
    """for id in df_picklist[df_index]:
        if id not in new_dic.keys():
            new_dic[df_index].append(id)""" 
            
    for column in set(picklist_dict.keys()):
        for value in df_picklist[column]:
            if value not in set(picklist_dict[column]): 
                if pd.isnull(value) or pd.isna(value) or value=='' or value=='nan':
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorrect value: {value}')
            else:
                new_dic[column].append('Ok!')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok!', np.nan)
    df_errors[df_index] = [i for i in df[df_index]]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()

    df = df[[df_index, df_status]]
    df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors[[df_status]+ df_errors.columns[:-1].tolist()]
    df_errors = df_errors.reset_index()
    return df_errors

picklist_tw_general = {
    'categorization by transmission sys': ['Macro', 'Macro+DAS', 'Public DAS','Transmission']
}

df_general_pick = check_picklist_v1(towerdb, tw_index,tw_status, picklist_tw_general)
df_general_pick

,active / not active
code,


In [ ]:
def check_mom_bts(df_tw, tw_index,status_col, bts_col, df_msa):

    msa_bts = df_msa[df_msa[bts_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[tw_index]]

    tw_bts = df_tw[df_tw[bts_col]=='Yes']
    tw_bts_sites = [i for i in tw_bts[tw_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = df_msa[df_msa[tw_index].isin([int(i) for i in out_tower_bts])]
    filtered = filtered[[tw_index,status_col, bts_col]]
    actual = df_tw[df_tw[tw_index].isin(out_tower_bts)][[tw_index, bts_col, status_col]]
    merge = pd.merge(filtered, actual, how='inner', on=tw_index, suffixes=['_old', '_in_month'])

    return  merge[[tw_index, status_col+'_old',status_col+'_in_month', bts_col+'_old',bts_col+'_in_month']]

actives = towerdb[towerdb[tw_status]=='In Service']
df_mom_bts = check_mom_bts(actives, tw_index,tw_status, tw_bts, msa, msa_index, msa_bts)
# No one error df_mom_bts

,code,active / not active,bts_site


Check Picklist and dates formats for In service sites



In [18]:
def check_picklist_v1(df,df_index,df_status, picklist_dict):
    from collections import defaultdict
    
    df_cols = list(picklist_dict.keys())
    df_picklist = df[df_cols].fillna('')
    df_picklist = df_picklist.to_dict(orient='list')

    new_dic = defaultdict(list)
    """for id in df_picklist[df_index]:
        if id not in new_dic.keys():
            new_dic[df_index].append(id)""" 
            
    for column in set(picklist_dict.keys()):
        for value in df_picklist[column]:
            if value not in set(picklist_dict[column]): 
                if pd.isnull(value) or pd.isna(value) or value=='' or value=='nan':
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorrect value: {value}')
            else:
                new_dic[column].append('Ok!')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok!', np.nan)
    df_errors[df_index] = [i for i in df[df_index]]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()

    df = df[[df_index, df_status]]
    df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors[[df_status]+ df_errors.columns[:-1].tolist()]
    df_errors = df_errors.reset_index()
    return df_errors

picklis_dict = {
    'confirmed skylon scope': ['In scope','Out of scope'],
    'categorization by transmission sys (subcluster)': ['Standard','Core','Dual Use Site','Transmission','Repeater','Long-term Mobile','DAS active',\
                                                        'DAS passive','Macro+DAS active','Macro+DAS passive','Outdoor Small Cells'],
    'categorization by transmission sys': ['Macro', 'Macro+DAS', 'Public DAS','Transmission','Repeater','Long-term Mobile', 'Outdoor Small Cells'],
    'categorization by site type': ['RTT','GBT','DAS active' ,'DAS passive' ,'RTT+DAS active' ,'GBT+DAS active','RTT+DAS passive' ,'GBT+DAS passive','Outdoor Small Cells'],
    'additional information - room configuration (including container information)': ['Indoor','Outdoor','Indoor & Outdoor'],
    'type of air cooling' : ['No','Yes; Indoor Air Conditioning','Yes; Indoor Air Conditioning and Free Air cooling / Free cooling units'],
    'core site type ': ['Case A','Case B','Non Core'],
    'macro site - transmission hub site': ['Yes', 'No'],
    'macro site - transmission hub site with/without shelters': ['With shelters','Without shelters','Non Transmission Hub Site'],
    'active sharing arrangements involving the operator': ['MORAN (On VF equipment)','MORAN (On non-VF equipment)','MOCN with Spectrum Pooling',\
                                                           'Partial Active-Passive','No Active Sharing'],         
    'critical site': ['Yes', 'No'],
    'strategic site': ['Yes', 'No'],
    'telenor tenant': ['Yes', 'No'],
    'telekom tenant': ['Yes', 'No'],
    'power supply for tenants': ['AC', 'DC', 'no power'],
    'energy category': ['X','Y','Z-1','Z-2','No energy consumption'],
    'wip_site': ['Yes', 'No'],
    'bts_site': ['Yes', 'No'],
    'strategic site': ['Yes', 'No'],
    'critical site': ['Yes', 'No'],
    'strategic_site_bucket': ['Yes - 0-5%','Yes - 5-10%','Non Strategic'],
    'critical_site_beyond_10':['Beyond 10%','Within 10%','Non Critical'],
    'sites_as_metered_estimated': ['Estimated Model','Metered Model']
}

actives_2 = towerdb[towerdb[tw_status]=='In Service']
df_in_service_picklist = check_picklist_v1(actives_2,tw_index,tw_status, picklis_dict)
df_in_service_picklist
#muitos errors


,code,active / not active,sites_as_metered_estimated,power supply for tenants,categorization by site type
0,1001,In Service,NaN,Blank Value,NaN
1,1002,In Service,NaN,Blank Value,NaN
2,1004,In Service,NaN,Blank Value,NaN
3,1005,In Service,NaN,Blank Value,NaN
4,1007,In Service,NaN,Blank Value,NaN
...,...,...,...,...,...
1826,RGU0352,In Service,NaN,Blank Value,Incorrect value: DAS Passive
1827,RGU0453,In Service,NaN,Blank Value,Incorrect value: DAS Passive
1828,RU0094,In Service,NaN,Blank Value,Incorrect value: DAS Passive
1829,RU0138,In Service,NaN,Blank Value,Incorrect value: DAS Passive


In [25]:
def check_date_columns(df, df_index,status_col,columns):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    from collections import defaultdict

    df_dates = df[columns].fillna('')
    df_dates = df_dates.to_dict(orient='list')
    #df_dates['sites'] = df_dates[df_index]
    #df_dates = df_dates.set_index('sites')
    new_dic = defaultdict(list)

    date_format = re.compile(r"[0-9]{1,2}[\_|\-|\/][0-9]{1,2}[\_|\-|\/][0-9]{1,4}")

    for column in set(df_dates.keys()):
        for value in df_dates[column]:
            if date_format.match(value) == None:
                if value in (None, '', 'nan'):
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorret picklist value: {value}')
            else:
                new_dic[column].append('Ok')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok', np.nan)
    df_errors[df_index] = [i for i in df[df_index]]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()

    df = df[[df_index, status_col]]
    df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors[[status_col]+ df_errors.columns[:-1].tolist()]
    df_errors = df_errors.reset_index()
    return df_errors

actives_1 = towerdb[towerdb[tw_status]=='In Service']
date_cols = ['infrastructure ready (existing)/ to be ready (new)', 'ready for active installation ("rfai") date', 'first_active_sharing_start_date','first_active_sharing_end_date']
#start_dates = ['infrastructure ready (existing)/ to be ready (new)', 'ready for active installation ("rfai") date', 'first_active_sharing_start_date','first_active_sharing_end_date']

#date_parser(actives, start_dates, "%d/%m/%Y", 'no')
df_in_service_dates = check_date_columns(actives_1, tw_index,tw_status, date_cols)
df_in_service_dates
#Muitos Campos em branco

,code,active / not active,first_active_sharing_start_date,"ready for active installation (""rfai"") date",first_active_sharing_end_date
0,1001,In Service,Blank Value,Blank Value,Blank Value
1,1002,In Service,Blank Value,Blank Value,Blank Value
2,1004,In Service,Blank Value,Blank Value,Blank Value
3,1005,In Service,Blank Value,Blank Value,Blank Value
4,1007,In Service,Blank Value,Blank Value,Blank Value
...,...,...,...,...,...
2048,RGU0352,In Service,Blank Value,Blank Value,Blank Value
2049,RGU0453,In Service,Blank Value,Blank Value,Blank Value
2050,RU0094,In Service,Blank Value,Blank Value,Blank Value
2051,RU0138,In Service,Blank Value,Blank Value,Blank Value


Fifth Check BTS Flagged(Billing Trigger and Commercial)

In [ ]:
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
        
    if t == 'doer':
        df_tw[date_col]= pd.to_datetime(df_tw[date_col], format='%d/%m/%Y', errors='coerce')
        filtered = df_tw[(df_tw[status_col]==status)&((df_tw[date_col] < current_date)|(df_tw[date_col]==''))]
        filtered[date_col] = list(map(lambda x: f'{x:%d/%m/%Y}',filtered[date_col]))
        return filtered[[tw_index, status_col, date_col]] 
    else:
        #df_tw[date_col]= pd.to_datetime(df_tw[date_col], format='%d/%m/%Y', errors='coerce')
        filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col]=='')]
        return filtered[[tw_index, status_col, date_col]] 

status = 'Yes'
actives = towerdb[towerdb[tw_index]=='In Service']
df_bts_empty = check_tw_bill_doer(actives, tw_index,tw_bill, tw_bts, status, 'bill')

#No errors

Check MoM Sites (BTS)

In [ ]:
""" BTS sites"""
path_uip = '/content/UserInput_Hungary_20210831.xlsx'
uip_names = ['Site_ID','BTS site applicable charge (Annual)',\
             'Commercials for sites beyond 10% cap of critical sites (Annual)']
uip = pd.read_excel(path_uip ,sheet_name='SiteLevel',usecols=[0,1,2],skiprows=2)
uip.columns = uip_names

msa_sites = [i for i in msa[msa_index]]
tw_sites = [i for i in towerdb[tw_index]]
uip_sites = [str(i) for i in uip['Site_ID']]

In [ ]:
""" BTS sites"""
def check_new_sites(df_towerdb, tw_index, bts_col, bill_col, status_col, msa_list, towerdb_list, uip_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    filtered = df_towerdb[[tw_index, status_col, bts_col]]

    #Finding for ALL NEW SITES
    new_site = [i for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    new_sites = pd.merge(new_sites, filtered, how='left', left_on=['New_Sites'], right_on=tw_index)
    new_sites = new_sites[['New_Sites', status_col, bts_col]]

    #create a copy to make index modifications
    df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    df[bill_col] = pd.to_datetime(df[bill_col], format='%d/%m/%Y', errors='coerce')
    df_site_bts = df[(df[bts_col]=='Yes')&(df[bill_col] > current_date)].fillna('')
    df_site_bts[bill_col] = list(map(lambda x: f'{x:%d/%m/%Y}', df_site_bts[bill_col]))
    df_site_bts = df_site_bts[[tw_index,status_col, bts_col, bill_col]]
    
    #if not (new_sites.empty or bts_out_uis.empty or df_site_bts.empty):
    return new_sites, df_site_bts[[tw_index,status_col, bts_col, bill_col]]

new_sites, df_bts_errors = check_new_sites(towerdb, tw_index, tw_bts, tw_bill,tw_status, msa_sites, tw_sites, uip_sites)

print(new_sites)
print(df_bts_errors)


Check UIP sites with towerDB
Some countries doesn't have decommissioned column

In [ ]:
""" Wip Sites Check"""
def check_wip(df_tw,tw_index, wip_tw, tw_bts, tw_status, df_msa, msa_index, wip_msa_col):

    wip_msa = [i for i in df_msa[df_msa[msa_wip]=='Yes'][msa_index]]
    
    tw_wip_sites = [str(i) for i in df_tw[df_tw[wip_tw]=='Yes'][tw_index]]

    tw_wip_site_bts_flagged = df_tw[(df_tw[wip_tw]=='Yes')&(df_tw[tw_bts]=='Yes')]
    tw_wip_site_bts_flagged = tw_wip_site_bts_flagged[[tw_index, wip_tw, tw_bts]]
    
    #wip_out_tw_list = [i for i in tw_wip_sites if i not in wip_msa]
    return tw_wip_site_bts_flagged

df_wip_and_bts_flagged = check_wip(towerdb,tw_index, tw_wip, tw_bts,tw_status, msa, msa_index, msa_wip)

print(df_wip_and_bts_flagged)


Empty DataFrame
Columns: [code, wip_site, bts_site]
Index: []


In [ ]:
#Coluna Doer tem valores fora do formato
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
        
    if t == 'doer':
        df_tw[date_col]= pd.to_datetime(df_tw[date_col], format='%d/%m/%Y', errors='coerce')
        filtered = df_tw[(df_tw[status_col]==status)&((df_tw[date_col] < current_date)|(df_tw[date_col]==''))]
        filtered[date_col] = list(map(lambda x: f'{x:%d/%m/%Y}',filtered[date_col]))
        return filtered[[tw_index, status_col, date_col]] 
    else:

        filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col]=='')]
        return filtered[[tw_index, status_col, date_col]] 

actives = towerdb[towerdb[tw_status]=='In Service']
df_doer = check_tw_bill_doer(actives, tw_index, tw_doer, tw_status, 'In Service', 'doer')


In [ ]:
#Retorna os sites novos
def check_bts(df_tw, bts_tw_columns, tw_index, status_col, df_msa, bts_msa_column, msa_index):

    bts_msa = [i for i in msa[msa[bts_msa_column]=='Yes'][msa_index]]
    tw_bts_sites = [i for i in df_tw[df_tw[bts_tw_columns]=='Yes'][tw_index]]
    #return of datas
    bts_out_tw = [i for i in bts_msa if i not in tw_bts_sites]
    filtered = df_msa.loc[df_msa[tw_index].isin(bts_out_tw), [tw_index, status_col, bts_tw_columns]]
    return filtered

df_bts_out = check_bts(towerdb, tw_bts, tw_index, tw_status, msa, msa_bts, msa_index)
df_bts_out

,code,active / not active,bts_site


In [ ]:
def check_uip_tw(df_tw,tw_index, status_tw_col, tw_bts_col, tw_critical_col, df_uip, uip_sites):

    filtered = df_tw[[tw_index, status_tw_col, tw_bts_col]]
    #tw_active = df_tw[df_tw['Site Status']=='In Service']
    count_tw_sites = [str(i) for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

    # check number of sites that are in uip file and doesn't have in df_tw
    #if not set(count_tw_sites).intersection(uip_sites):
    uis_sites_not_in_towerdb = [i for i in uip_sites if i not in count_tw_sites]

    uis_sites_not_in_towerdb = pd.DataFrame(uis_sites_not_in_towerdb,columns=['UIS In Month not active in TowerDB!'])
    uis_sites_not_in_towerdb = pd.merge(uis_sites_not_in_towerdb, filtered, how='left', left_on='UIS In Month not active in TowerDB!',\
                                    right_on=tw_index)
    uis_sites_not_in_towerdb = uis_sites_not_in_towerdb[['UIS In Month not active in TowerDB!', status_tw_col]]

    in_service_not_in_uis = [i for i in count_tw_sites if i not in uip_sites]
    in_service_not_in_uis = pd.DataFrame(in_service_not_in_uis,columns=['TowerDB Sites out of UIS In Month!'])
    in_service_not_in_uis = pd.merge(in_service_not_in_uis, filtered, how='left', left_on='TowerDB Sites out of UIS In Month!',\
                                    right_on=tw_index)
    in_service_not_in_uis = in_service_not_in_uis[['TowerDB Sites out of UIS In Month!', status_tw_col]]
    #check for decomissioned site not in uip files

    #Check BTS sites
        #Check BTS sites
    bts_sites = [str(i) for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
    uip_bts = [i for i in df_uip[df_uip['BTS site applicable charge (Annual)']!='']['Site_ID']]
    bts_sites_out_uip = [i for i in uip_bts if i not in bts_sites]
    bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['UIS BTS not in TowerDB(BTS)'])
    bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='UIS BTS not in TowerDB(BTS)',\
                                    right_on=tw_index)
    bts_sites_out_uip = bts_sites_out_uip[['UIS BTS not in TowerDB(BTS)', status_tw_col, tw_bts_col]]

    #  Check for UIP critical sites 
    uip_critical = [i for i in df_uip[df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!=0]['Site_ID']]
    bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Yes'][tw_index]

    critical = [i for i in uip_critical if i not in bts_tw_critical]
    critical = pd.DataFrame(critical, columns=['Critical site(beyond 10%) out of UIS'])
    critical = pd.merge(critical, filtered, how='left', left_on='Critical site(beyond 10%) out of UIS',right_on=tw_index)
    critical = critical[['Critical site(beyond 10%) out of UIS', status_tw_col]]
    #if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
    return uis_sites_not_in_towerdb, in_service_not_in_uis, bts_sites_out_uip, critical
   
#(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites)
uis_sites_not_in_towerdb, in_service_not_in_uis, bts_sites_out_uip, critical = check_uip_tw(towerdb,tw_index, tw_status, tw_bts,tw_critical,\
                                                                 uip, uip_sites)
print(uis_sites_not_in_towerdb)
print('\n')
print(in_service_not_in_uis)
print('\n')
print(critical)
#no errors

Commercial

In [ ]:
def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan']
        #lista = []

        df.fillna("", inplace=True)
        #df[column] = df[column].astype('int64')

        for column in columns:
            lista = []
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista

        return df    
    
    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names).fillna('')
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names ).fillna('')
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols).fillna('')

    df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    if not df_commercial_diffs.empty:
        return df_commercial_diffs
    else:
        print('\nNo errors founded!')

path_before = '/content/UserInput_Hungary_20210731.xlsx'
names = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', 'Input_Value',\
        'Description/Instruction', 'Frequency of Update']
merge_cols = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', \
              'Description/Instruction', 'Frequency of Update']
cols_ordered = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2','Data_Type','Input_Value_actual',\
                'Input_Value_before','Equal Values','Description/Instruction','Frequency of Update']
col_replaced = ['Input_Value']
df_commercial_diffs = check_commercial(path_uip, path_before,col_replaced, names, merge_cols, cols_ordered)
#df_commercial_diffs = check_commercial(path_uip, path_before, replace_values, names, merge_cols, cols_ordered)


Excel Log

In [ ]:
def general_log_erros(df_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')  
    
    for lista in df_list:
        try:
            if not lista[1].empty:
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
        except AttributeError:
            pass
        except:
            if lista[1] is not None and lista[1].is_valid():
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
    try: 
        writer.save() 
    except IndexError:
        print('No one log to write!')

df_list = [
    ['In Service Date Errors',actives_dates_errors],
    ['Dismantled Dates Errors',no_actives_dates_errors],
    ['General Picklist Errors',df_general_pick],
    ['In Service Picklist Errors',df_in_service_picklist],
    ['Actives Blank Dates',df_in_service_dates],
    ['Bill Dates Errors',df_bts_empty],
    ['New Sites',new_sites],
    ['New Sites Bill date errors',df_bts_errors],
    ['WIP & BTS Flagged',df_wip_and_bts_flagged],
    ['DOER Dates Errors',df_doer],
    ['UIS In Month not active',uis_sites_not_in_towerdb],
    ['TowerDB Sites out of UIS',in_service_not_in_uis],
    ['Critical Sites out UIS',critical],
    ['Commercial Differences',df_commercial_diffs]]
path = '/content/HU_towerdb_errors.xlsx'
general_log_erros(df_list, path)

TA Input

In [ ]:
def replace_values(df, columns, value=""):
    """
    Está voltando para float
    """
    invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ']

    for column in columns:
        lista = []
        df[column] = df[column].fillna(0)
        for index in df[column]:
            #print(f"{column} -> {index}")
            if index in invalid_values:
                lista.append(value)
            else:
                lista.append(index)
        df[column] = lista
    return df

def date_parser(df, columns,format='mix', type_date="%d/%m/%Y"):
    for column in columns:
        if format == 'mix':
            df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                        and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
            df[column] = df[column].astype(str)
        else:
            df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
            df[column] = df[column].astype(str)
ta_ord = ['SiteID','SiteID.1','In/out of scope','Tenant Agreement ID','Agreement relates to DAS site?','Tenant (name/ID)',\
          'Classification','MNO classification','Base fee (@ signing) in HUF','Additional fee (@ signing) in HUF',\
          'Annual Fee per Tenant (@ signing) in HUF','Annual Energy Fee  (@ signing)','Annual Maintenance Fee  (@ signing)',\
          'Other Services Fee  (@ signing)','Base fee in HUF','Additional fee in HUF','Current Annual Fee per Tenant in HUF',\
          'Current Annual Energy Fee in HUF','Starting date','Expiring date (raw data)','Expiring date (in practice)',\
          'Renewal Option','Expiring date after renewal','Terms of payments','Payment type','Index','Percentage','Indexation driver',
          'VAT Suject','Percentage (VAT)','Termination Date','Residual duration']
          
ta_leases = ['Tenant Agreement ID','MNO classification', 'Base fee (@ signing) in HUF', 'Starting date',
       'Expiring date (raw data)', 'Expiring date (in practice)']
dates_ta = ['Starting date','Expiring date (raw data)', 'Expiring date (in practice)']
pathta = '/content/TA_Input_Hungary_20210831.csv'
ta = pd.read_csv(pathta,  encoding='latin').fillna('')
#ta = replace_values(ta,ta_leases, '')
#i != '0' or i!= "" and not 
lista = []
for i in ta['Base fee (@ signing) in HUF']:
    cur = re.compile('\d{1,3}(?:,\d{3})')
    if cur.match(str(i)) == None :
        new_i = int(i)
        new_i = f'{new_i:,d}'
        lista.append(new_i)
    else:
        lista.append(i)
ta['Base fee (@ signing) in HUF'] = lista

#No erros
for col in ['Starting date','Expiring date (raw data)', 'Expiring date (in practice)']:
    ta[col] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) \
                        and not isinstance(date_obj, str) else date_obj for date_obj in ta[col]]
ta['Expiring date (raw data)'] = ta['Expiring date (raw data)'].fillna(0)
ta['Expiring date (raw data)'] = ta['Expiring date (raw data)'].replace(0, '')
ta['Payment type'] = ta['Payment type'].replace('', 0).astype(int)
ta = ta[ta_ord]

ta = ta.rename(columns={'SiteID.1':'SiteID'})
ta.to_csv('/content/TA_Input_Hungary_20210831_celfinet.csv', index=False, encoding='latin')
ta.head(2)

In [ ]:
import datetime as dt
#Create dates in datetime format
lista = []
for i in ta['Expiring date (raw data)']:
    try:
        if i != "":
            lista.append(pd.to_datetime(i, format='%d/%m/%Y'))
        else:
            lista.append(datetime(1999, 1, 1, 0, 0, 0))
    except:
        lista.append(datetime(2199, 1, 1, 0, 0, 0))

ta['end'] = lista
ta['start'] = [pd.to_datetime(i, format='%d/%m/%Y') if i!= '' else datetime(1999, 1, 1, 0, 0, 0) for i in ta['Starting date']]
#Create diff columns to get negative values
ta['dt_dff'] = (ta['end'] - ta['start']).dt.days

start_or_end_blank = (ta['Starting date']=='')|(ta['Expiring date (raw data)']=='')
negative_dates = (ta['dt_dff'] <= 0)
cols = ['SiteID', 'Starting date', 'Expiring date (raw data)']
ta_dates_errors = ta[start_or_end_blank|negative_dates][cols]
ta_dates_errors

,SiteID,SiteID,Starting date,Expiring date (raw data)


In [ ]:
def check_amounts(df, df_index, columns, pattern=','):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    from collections import defaultdict

    df_am = df[columns].fillna('')
    df_am = df_am.to_dict(orient='list')

    new_dic = defaultdict(list)

    for column in set(df_am.keys()):
        for value in df_am[column]:
            if not str(value).__contains__(pattern):
                if value in (None, '', 'nan'):
                    new_dic[column].append('Blank Value')
                elif value == '0':
                    new_dic[column].append('Input 0')
                else:
                    new_dic[column].append(f'Incorret picklist value: {value}')
            else:
                new_dic[column].append('Ok')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok', np.nan)
    df_errors[df_index] = df[df_index]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()
    #df_errors = df_errors.reset_index()
    return df_errors

#Drop duplicate of SiteID
ta = ta.T.drop_duplicates().T
ta_amount = check_amounts(ta, 'SiteID',['Base fee (@ signing) in HUF'])
ta_amount

In [ ]:
df_class_error = ta[(ta['MNO classification']=='')|~(ta['MNO classification'].isin(['MNO 1','MNO 2','MNO 3', 'OTMO']))][['SiteID', 'MNO classification']]
df_class_error

Ta Log

In [ ]:
def general_log_erros(df_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')  
    
    for lista in df_list:
        try:
            if not lista[1].empty:
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
        except AttributeError:
            pass
        except:
            if lista[1] is not None and lista[1].is_valid():
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
    try: 
        writer.save() 
    except IndexError:
        print('No one log to write!')

ta_s = [['Dates Errors',ta_dates_errors],
        ['Amount Errors', ta_amount],
        ['Picklist Errors', df_class_error]]
path_ta_log = '/content/TA_HU_errors.xlsx'
general_log_erros(ta_s, path_ta_log)

LC Input

In [ ]:
 #separador virgulas
pathlc = '/content/LC_Input_Hungary_20210831.csv'

lc_ord = ['Code','Code.1','In/out of scope','Lease Contract ID','Counterpart','Annual lease fees (@ signing) in HUF',
          'Annual lease fees (@ signing) in Original currency','Contract currency','Energy fee (@ signing)',
          'Other fees  (@ signing)','Current annual lease fees in HUF','Current annual lease fees in original curency',
          'Contract currency','Current Energy fee','Current other fees','Sub-lease','# of renegotiation','Starting date',
          'Expiring date (raw data)','Expiring date (in practice)','Renewal Option','Expiring date after renewal',
          'Terms of payments (days)','Payment type (number of payments per year)','Some period already prepaid','Indexation',
          'Indexation Percentage','Indexation driver','VAT Suject','Percentage (VAT)','Termination Date','# of years of renewal',
          '# of months of renewal','Type of contrac','Lease renewal (years)','Residual duration']

lc = pd.read_csv(pathlc,  encoding='latin').fillna('')
lc = lc[lc_ord]
leases = ['Current annual lease fees in HUF']
lc = replace_values(lc,leases, '')

lc_cols = ['Code', 'Current annual lease fees in HUF'] 

lc = lc.rename(columns={'Code.1': 'Code'})
lc.to_csv('/content/LC_Input_Hungary_20210831_celfinet.csv', index=False, encoding='latin')
lc.head(2)
#No erros

In [ ]:
def check_amounts(df, df_index, columns, pattern=','):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    from collections import defaultdict

    df_am = df[columns].fillna('')
    df_am = df_am.to_dict(orient='list')

    new_dic = defaultdict(list)

    for column in set(df_am.keys()):
        for value in df_am[column]:
            if not str(value).__contains__(pattern):
                if value in (None, '', 'nan'):
                    new_dic[column].append('Blank Value')
                elif value == '0':
                    new_dic[column].append('Input 0')
                else:
                    new_dic[column].append(f'Incorret picklist value: {value}')
            else:
                new_dic[column].append('Ok')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok', np.nan)
    df_errors[df_index] = df[df_index]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()
    #df_errors = df_errors.reset_index()
    return df_errors

#Drop duplicate of SiteID
lc = lc.T.drop_duplicates().T
df_lc_amount = check_amounts(lc, 'Code', leases)
df_lc_amount


In [ ]:
import datetime as dt
#Create dates in datetime format
lista = []
for i in lc['Expiring date (raw data)']:
    try:
        if i != "":
            lista.append(pd.to_datetime(i, format='%d/%m/%Y'))
        else:
            lista.append(datetime(1999, 1, 1, 0, 0, 0))
    except:
        lista.append(datetime(2199, 1, 1, 0, 0, 0))

lc['end'] = lista
lc['start'] = [pd.to_datetime(i, format='%d/%m/%Y') if i!= '' else datetime(1999, 1, 1, 0, 0, 0) for i in lc['Starting date']]
#create diff columns to get negative values
lc['dt_dff'] = (lc['end'] - lc['start']).dt.days

start_or_end_blank = (lc['Starting date']=='')|(lc['Expiring date (raw data)']=='')
negative_dates = (lc['dt_dff'] <= 0)
cols = ['Code', 'Starting date', 'Expiring date (raw data)']
lc_dates_errors = lc[start_or_end_blank|negative_dates][cols]

df_lc_amount
lc_dates_errors

,Code,Starting date,Expiring date (raw data)


In [ ]:
def general_log_erros(df_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')  
    
    for lista in df_list:
        try:
            if not lista[1].empty:
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
        except AttributeError:
            pass
        except:
            if lista[1] is not None and lista[1].is_valid():
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
    try: 
        writer.save() 
    except IndexError:
        print('No one log to write!')

df_lc = [['Amount Errors', df_lc_amount],
         ['Dates Errors', lc_dates_errors]]

general_log_erros(df_lc,'/content/LC_HU_errors.xlsx')

UIS Comparioson